In [7]:
# 딱 한번만 돌릴것
from CrawlingModule import YoutubeBulider
# KEY 파일로 읽어오기
KEY_FILE = open('./API_KEY.txt', 'r')
# API_KEY 할당
API_KEY = KEY_FILE.readline()

# YoutbeModule 사용
bulider = YoutubeBulider(API_KEY)

channelId = bulider.search_channelId('김성회의 G식백과')
video_id_list = bulider.get_videoId_in_channel(channelId=channelId)

In [8]:
import pandas as pd

In [9]:
category_id_list, view_count_list = bulider.get_categoryId_in_channel(videoId_list=video_id_list)

TypeError: list.append() takes exactly one argument (2 given)

In [10]:
df = pd.DataFrame(view_count_list, columns=['video_id', 'view_count', 'like_count'])
df

,view_count,like_count
0,761102,9227
1,605330,8244
2,168117,3969
3,58431,528
4,38362,477
...,...,...
295,638938,9629
296,118602,2600
297,514631,6606
298,234568,3284


In [181]:
df = pd.DataFrame(
    {
        'video_id' : video_list, 
        'category_id' : list(map(int, category_id_list)),
    }
)
# df.category_id = df.category_id.astype(int)


In [217]:
infile = open('./category_id_list.txt', 'r').readlines()

key_value_dict = {}

for line in infile:
    key_value_list = line.split(' - ')
    key_value_dict[int(key_value_list[0])] = key_value_list[1][:-1]


df['category_name'] = df.category_id.apply(lambda x : key_value_dict[x])

In [219]:
df.to_csv('./데이터/videoId_and_categoryId_lists.csv', index=False)

In [60]:
df.category_id.value_counts()

,video_id,category_id,category_name
0,Ynwm92g2hxQ,20,20
1,a4abVmaMj5A,20,20
2,TUTMzIP_Mv0,20,20
3,zrN-ybe49lg,20,20
4,PuRqyxLxTTc,20,20
...,...,...,...
95,u9zKB3s-R1A,20,20
96,De0mePZH0rI,20,20
97,Bb43QlfkCAE,20,20
98,wEa_8r0R3P8,20,20
